In [10]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 26 13:02:19 2019

@author: raiak
"""

import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
from datetime import date
import datetime

today = date.today()
today=date.strftime(today,'%d/%m/%Y')
urlheader = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
filename=['BrentCrude.csv','DowHistorical.csv','FTSE_Historical.csv','HangSeng_Historical.csv','India VIX Historical Data.csv',
          'USD_INR_Historical.csv']
url = "https://in.investing.com/instruments/HistoricalDataAjax"


for each in filename:
    with open(each, 'r') as readFile:
        reader = csv.reader(readFile)
        lines = list(reader)
    startdate=lines[1][0]
    startdate=datetime.datetime.strptime(startdate,'%d-%b-%y')
    startdate += datetime.timedelta(days=1)
    startdate=startdate.strftime('%d/%m/%Y')
    readFile.close()
    if (each == 'BrentCrude.csv'):
        header="Brent Oil Futures Historical Data"
        curr_id="8833"
        smlID= "300028"
        column_name='Brent Crude Price'
    elif (each == 'DowHistorical.csv'):
        header="Dow Jones Industrial Average Historical Data"
        curr_id="169"
        smlID= "2030170"
        column_name='Dow Price'
    elif (each == 'FTSE_Historical.csv'):
        header="FTSE 100 Historical Data"
        curr_id="27"
        smlID= "2030166"
        column_name='FTSE'
    elif (each == 'HangSeng_Historical.csv'):
        header="Hang Seng Historical Data"
        curr_id="179"
        smlID= "2030179"
        column_name='Hang Seng Price'
    elif (each == 'India VIX Historical Data.csv'):
        header="India VIX Historical Data"
        curr_id="17942"
        smlID= "2036406"
        column_name='Volatility Index VIX'
    elif (each == 'USD_INR_Historical.csv'):
        header="USD/INR Historical Data"
        curr_id="160"
        smlID= "106815"
        column_name='USD-INR price'
 
    payload = {'header': header, 
               'st_date': startdate, 'end_date': today, 
               'sort_col': 'date', 'action': 'historical_data', 
               'smlID': smlID, 'sort_ord': 'DESC', 'interval_sec': 'Daily', 'curr_id': curr_id}

    req = requests.post(url, headers=urlheader, data=payload)
    soup = BeautifulSoup(req.content, "lxml")
    table = soup.find('table', id="curr_table")
    split_rows = table.find_all("tr")
    rows = table.findAll('tr')
    header_text = []
    headers = rows[0]
    
    # add the header text to array
    for th in headers.findAll('th'):
        header_text.append(th.text)
        
    del header_text[2:]
    
    row_text_array = []
    for row in rows[1:]:
        row_text = []
        # loop through the elements
        for row_element in row.findAll(['th', 'td']):
            # append the array with the elements inner text
            row_text.append(row_element.text.replace('\n', '').strip())
        del row_text[2:]
        # append the text array to the row text array
        row_text_array.append(row_text)
    
    df = pd.read_csv(each)
    
    df1 = pd.DataFrame(row_text_array)
    
    df1.rename(columns={0:'Date',1:column_name}, inplace=True)
    df=df.append(df1,sort=False)
    df=df.dropna()
    df['Date']= pd.to_datetime(df['Date'])
    df = df.sort_values('Date',ascending=False)
    
    df['Date']= df['Date'].dt.strftime('%d-%b-%y')
    print(df.head(3))
    df.to_csv(each,index=False)

        Date  Brent Crude Price
0  16-Sep-19              65.50
1  15-Sep-19              68.41
2  13-Sep-19              60.22
        Date  Dow Price
0  16-Sep-19  27,110.89
1  13-Sep-19  27,219.52
2  12-Sep-19  27,182.45
        Date      FTSE
0  16-Sep-19  7,352.98
1  13-Sep-19  7,367.46
2  12-Sep-19  7,344.67
        Date Hang Seng Price
0  16-Sep-19       27,124.55
1  13-Sep-19       27,352.69
2  12-Sep-19       27,087.63
        Date  Volatility Index VIX
0  16-Sep-19                 14.93
1  13-Sep-19                 14.12
2  12-Sep-19                 14.90
        Date  USD-INR price
0  16-Sep-19         71.544
1  13-Sep-19         70.985
2  12-Sep-19         70.967
